This notebook is used to prepare amenity data (grocery stores) into distance data that can be used for clustering and visualization.



---

### Imports

In [1]:
import os
import json
import pickle
from collections import defaultdict
import gzip

import pandas as pd
import numpy as np
import scipy.spatial as spatial

import geopandas as gp
from shapely.geometry import Point, Polygon

from IPython.display import display

### Define working directory

In [2]:
ROOT = '/media/school/project/amenities'

### Read Gazetteer table

GEOIDS and lat/long for tract centers are pulled from this table.

In [3]:
gaz = pd.read_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_GAZ_v3.pkl'))
gaz.GEOID = gaz.GEOID.astype(int)
gaz.columns = [x.strip() for x in gaz.columns]
gaz.head(1)

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338


### Read Geopandas dataframe

This is not really used for processing but gives some evidence of sanity-checks.

In [4]:
with gzip.GzipFile(os.path.join(ROOT, 'all_census_tract_shapes.json.gz'), 'r') as f:
    TRACT_ALL = json.loads(f.read().decode('utf-8'))
gpdf = gp.GeoDataFrame.from_features(TRACT_ALL['features'])
gpdf.GEOID = gpdf.GEOID.astype(int)
gpdf.head(1)

,geometry,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,"POLYGON ((-93.16468 30.21663, -93.16392 30.216...",22,019,980000,1400000US22019980000,22019980000,9800,CT,5398742,2339


### Read Amenities Data

In [5]:
AMENITIES = {}
amenity_src_path = '../../amenities/source-data'

for amenity_file in os.listdir(amenity_src_path):
    print(f'Reading {amenity_file}')
    name = amenity_file.split('.')[0].strip().replace('-', '_').split('_')[0].upper()
    data = pd.read_csv(os.path.join(amenity_src_path, amenity_file))
    AMENITIES[name] = data
    
print(AMENITIES.keys(), end='\n\n')
for k, v in AMENITIES.items():
    print(f'{k} shape: {v.shape}')

Reading grocery-stores.csv
Reading gyms.csv
dict_keys(['GROCERY', 'GYMS'])

GROCERY shape: (53518, 11)
GYMS shape: (49348, 11)


### Prepare the first fiew fields of the `result` table.

This will be used to aggregate results throughout the code.

In [6]:
# result with start off with the gaz GEOID and lat/long columns
result = gaz[['GEOID', 'INTPTLAT', 'INTPTLONG']]
result.head()

,GEOID,INTPTLAT,INTPTLONG
0,1001020100,32.481959,-86.491338
1,1001020200,32.475758,-86.472468
2,1001020300,32.474024,-86.459703
3,1001020400,32.471030,-86.444835
4,1001020500,32.458922,-86.421826


The following is a test to see if all geoids in gaz have corresponding geometries in gpdf.

219 in gaz are not in gpdf. These are tracts that correspond to waterways (bays, lakes, etc). A few were spot checked, for the rest, it was confirmed that ALAND was 0 in 218 records. The last record has ALAND > 0, but corresponds to tampa bay.

In [7]:
if not os.path.exists(os.path.join(ROOT, 'gaz_to_gpdf_geoid_mapping.pickle')):
    gaz_to_gpdf_geoid_mapping = {}
    for i, row in result.iterrows():
        print('\r{} '.format(i+1), end='')
        geoid = row.GEOID
        lat = float(row.INTPTLAT)
        lon = float(row.INTPTLONG)
        pt = Point(lon, lat) # order here is important!!

        # set default mapping when point not found within geometry
        gaz_to_gpdf_geoid_mapping[geoid] = None

        # if there is a matching geoid, set that
        gpdf_geoid_match = gpdf[gpdf.GEOID.isin([geoid])]
        if gpdf_geoid_match.shape[0] == 1:
            gaz_to_gpdf_geoid_mapping[geoid] = gpdf_geoid_match.GEOID.values[0]
            print('FOUND')
        # otherwise, find if it falls in a geometry
        else:
            for j, row_gpdf in gpdf.iterrows():          
                poly = row_gpdf.geometry
                found_flag = None
                if pt.within(poly):
                    print('FOUND, extra processing')
                    found_flag = True
                    gaz_to_gpdf_geoid_mapping[geoid] = row_gpdf.GEOID
                    break
            if not found_flag:
                print('===NOT FOUND===')
    with open(os.path.join(ROOT, 'gaz_to_gpdf_geoid_mapping.pickle'), 'wb') as f:
        pickle.dump(gaz_to_gpdf_geoid_mapping, f, protocol=4)
else:
    with open(os.path.join(ROOT, 'gaz_to_gpdf_geoid_mapping.pickle'), 'rb') as handle:
        gaz_to_gpdf_geoid_mapping = pickle.load(handle)    

In [8]:
len([x for x,y in gaz_to_gpdf_geoid_mapping.items() if y is not None])

72837

In [9]:
len([x for x,y in gaz_to_gpdf_geoid_mapping.items() if y is None])

219

In [10]:
gaz_geo_set = set(gaz.GEOID)
gpdf_geo_set = set(gpdf.GEOID)
print('Number of gaz geoids in gpdf: {}'.format(
    len(gaz_geo_set.intersection(gpdf_geo_set))))
print('Number of gaz geoids NOT in gpdf: {}'.format(
    gaz.shape[0] - len(gaz_geo_set.intersection(gpdf_geo_set))))

Number of gaz geoids in gpdf: 72837
Number of gaz geoids NOT in gpdf: 219


Given that these are equivalent, we can say it is likely that there are no geometries for the bays, lakes, etc that fall in the 219.

### For each business in each amenity, identify the census tract geoid in which it falls.

In [ ]:
def find_within_census_tract(df):
    geoid_matches = []

    # create a dict of subsets of a merged dataframe for faster processing below
    gaz_gpdf = gaz.merge(gpdf, how='left', on='GEOID')
    gaz_gpdf_subset_dict = {}
    for state in gaz_gpdf.USPS.unique():
        gaz_gpdf_subset_dict[state] = gaz_gpdf[gaz_gpdf.USPS==state]

    for i, row in df.iterrows():
        print('\r{} '.format(i+1, end=''))
        pt = Point(row.lon_cleaned, row.lat_cleaned)
        state = row.state

        # subset to state to reduce search space
        gaz_gpdf_subset = gaz_gpdf_subset_dict[state]
        geoid_match = None # set default as None for no match
        # iterate through geometries
        for j, row_j in gaz_gpdf_subset.iterrows():
            # update geoid_match if found
            poly = row_j.geometry
            if not poly:
                continue
            if pt.within(poly):
                print('FOUND')
                geoid_match = row_j.GEOID
                break

        # append geoid_match
        geoid_matches.append(geoid_match)

        if not geoid_match:
            print('===NOT FOUND===')
    
    return geoid_matches


for name, df in AMENITIES.items():
    temp_name = f'{name}.stage1.pkl'
    temp_path = os.path.join(ROOT, temp_name)
    if os.path.exists(temp_path):
        print(f'Loading existing data at: {temp_path}')
        df = pd.read_pickle(temp_path)
        display(df.head())
        AMENITIES[name] = df
    else:
        print(f'Working on {name} amenity data. Finding GEOID in which each store falls.')
        geoid_matches = find_within_census_tract(df)
        df[f'{name}_GEOID_MATCH'] = geoid_matches
        df.to_pickle(os.path.join(ROOT, temp_name), protocol=4)        

Loading existing data at: /media/school/project/amenities/GROCERY.stage1.pkl


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,GROCERY_GEOID_MATCH
0,60415,Chicago Ridge,IL,ALDI,:arlem & Southwest Hwy,":arlem & Southwest Hwy,60415",60415,41.700313,41.70031,-87.797054,-87.79705,1.703182e+10
1,40219,Louisville,KY,B & E Salvage Store,.,".,40219",40219,38.128021,38.12802,-85.679443,-85.67944,2.111101e+10
2,79106,Amarillo,TX,Butch,.,".,79109",79109,35.190903,35.19090,-101.845847,-101.84585,4.837501e+10
3,60901,Kankakee,IL,Jewl,...,"...,60914",60914,41.186157,41.18616,-87.893356,-87.89336,1.709101e+10
4,54017,New Richmond,WI,Indomaret,@ Fresh market,"@ Fresh market,54017",54017,45.119296,45.11930,-92.537666,-92.53767,5.510912e+10
...,...,...,...,...,...,...,...,...,...,...,...,...
53513,29801,Aiken,SC,Bi-Lo,York Street,"York Street,not_available",not_available,33.579133,33.57913,-81.703930,-81.70393,4.500302e+10
53514,95336,Manteca,CA,WIC,Yosemite Ave,"Yosemite Ave,not_available",not_available,37.797921,37.79792,-121.218739,-121.21874,6.077005e+09
53515,15221,Pittsburgh,PA,ALDI,Yost Blvd,"Yost Blvd,15221",15221,40.425969,40.42597,-79.848442,-79.84844,4.200356e+10
53516,64093,Warrensburg,MO,Bi-Lo,Young Street,"Young Street,not_available",not_available,38.771079,38.77108,-93.735348,-93.73535,2.910196e+10


Working on GYMS amenity data. Finding GEOID in which each store falls.
1 
FOUND
2 
FOUND
3 
FOUND
4 
FOUND
5 
FOUND
6 
FOUND
7 
FOUND
8 
FOUND
9 
FOUND
10 
FOUND
11 
FOUND
12 
FOUND
13 
FOUND
14 
FOUND
15 
FOUND
16 
FOUND
17 
FOUND
18 
FOUND
19 
FOUND
20 
FOUND
21 
FOUND
22 
FOUND
23 
FOUND
24 
FOUND
25 
FOUND
26 
FOUND
27 
FOUND
28 
FOUND
29 
FOUND
30 
FOUND
31 
FOUND
32 
FOUND
33 
FOUND
34 
FOUND
35 
FOUND
36 
FOUND
37 
FOUND
38 
FOUND
39 
FOUND
40 
FOUND
41 
FOUND
42 
FOUND
43 
FOUND
44 
FOUND
45 
FOUND
46 
FOUND
47 
FOUND
48 
FOUND
49 
FOUND
50 
FOUND
51 
FOUND
52 
FOUND
53 
FOUND
54 
FOUND
55 
FOUND
56 
FOUND
57 
FOUND
58 
FOUND
59 
FOUND
60 
FOUND
61 
FOUND
62 
FOUND
63 
FOUND
64 
FOUND
65 
FOUND
66 
FOUND
67 
FOUND
68 
FOUND
69 
FOUND
70 
FOUND
71 
FOUND
72 
FOUND
73 
FOUND
74 
FOUND
75 
===NOT FOUND===
76 
FOUND
77 
FOUND
78 
FOUND
79 
FOUND
80 
FOUND
81 
FOUND
82 
FOUND
83 
FOUND
84 
FOUND
85 
FOUND
86 
FOUND
87 
FOUND
88 
FOUND
89 
FOUND
90 
FOUND
91 
FOUND
92 
FOUND
93 
FOUN

FOUND
744 
FOUND
745 
FOUND
746 
FOUND
747 
FOUND
748 
FOUND
749 
FOUND
750 
FOUND
751 
FOUND
752 
FOUND
753 
FOUND
754 
FOUND
755 
FOUND
756 
FOUND
757 
FOUND
758 
FOUND
759 
FOUND
760 
FOUND
761 
FOUND
762 
FOUND
763 
FOUND
764 
FOUND
765 
FOUND
766 
FOUND
767 
FOUND
768 
FOUND
769 
FOUND
770 
FOUND
771 
FOUND
772 
FOUND
773 
FOUND
774 
FOUND
775 
FOUND
776 
FOUND
777 
FOUND
778 
FOUND
779 
FOUND
780 
FOUND
781 
FOUND
782 
FOUND
783 
FOUND
784 
FOUND
785 
FOUND
786 
FOUND
787 
FOUND
788 
FOUND
789 
FOUND
790 
FOUND
791 
FOUND
792 
FOUND
793 
FOUND
794 
FOUND
795 
FOUND
796 
FOUND
797 
FOUND
798 
FOUND
799 
FOUND
800 
FOUND
801 
FOUND
802 
FOUND
803 
FOUND
804 
FOUND
805 
FOUND
806 
FOUND
807 
FOUND
808 
FOUND
809 
FOUND
810 
FOUND
811 
FOUND
812 
FOUND
813 
FOUND
814 
FOUND
815 
FOUND
816 
FOUND
817 
FOUND
818 
FOUND
819 
FOUND
820 
FOUND
821 
FOUND
822 
FOUND
823 
FOUND
824 
FOUND
825 
FOUND
826 
FOUND
827 
FOUND
828 
FOUND
829 
FOUND
830 
FOUND
831 
FOUND
832 
FOUND
833 
FOUND
834 

FOUND
1449 
===NOT FOUND===
1450 
FOUND
1451 
FOUND
1452 
FOUND
1453 
FOUND
1454 
FOUND
1455 
FOUND
1456 
FOUND
1457 
FOUND
1458 
FOUND
1459 
FOUND
1460 
FOUND
1461 
FOUND
1462 
FOUND
1463 
FOUND
1464 
FOUND
1465 
FOUND
1466 
FOUND
1467 
FOUND
1468 
FOUND
1469 
FOUND
1470 
FOUND
1471 
FOUND
1472 
FOUND
1473 
FOUND
1474 
FOUND
1475 
FOUND
1476 
FOUND
1477 
FOUND
1478 
FOUND
1479 
FOUND
1480 
FOUND
1481 
FOUND
1482 
FOUND
1483 
FOUND
1484 
FOUND
1485 
FOUND
1486 
FOUND
1487 
FOUND
1488 
FOUND
1489 
FOUND
1490 
FOUND
1491 
FOUND
1492 
FOUND
1493 
FOUND
1494 
FOUND
1495 
FOUND
1496 
FOUND
1497 
FOUND
1498 
FOUND
1499 
FOUND
1500 
FOUND
1501 
FOUND
1502 
FOUND
1503 
FOUND
1504 
FOUND
1505 
FOUND
1506 
FOUND
1507 
FOUND
1508 
FOUND
1509 
FOUND
1510 
FOUND
1511 
FOUND
1512 
FOUND
1513 
FOUND
1514 
FOUND
1515 
FOUND
1516 
FOUND
1517 
FOUND
1518 
FOUND
1519 
FOUND
1520 
FOUND
1521 
FOUND
1522 
FOUND
1523 
FOUND
1524 
FOUND
1525 
FOUND
1526 
FOUND
1527 
FOUND
1528 
FOUND
1529 
FOUND
1530 
FOUND


...

FOUND
13413 
FOUND
13414 
FOUND
13415 
FOUND
13416 
FOUND
13417 
FOUND
13418 
FOUND
13419 
FOUND
13420 
FOUND
13421 
FOUND
13422 
FOUND
13423 
FOUND
13424 
FOUND
13425 
FOUND
13426 
FOUND
13427 
FOUND
13428 
FOUND
13429 
FOUND
13430 
FOUND
13431 
FOUND
13432 
FOUND
13433 
FOUND
13434 
FOUND
13435 
FOUND
13436 
FOUND
13437 
FOUND
13438 
FOUND
13439 
FOUND
13440 
FOUND
13441 
FOUND
13442 
FOUND
13443 
FOUND
13444 
FOUND
13445 
FOUND
13446 
FOUND
13447 
FOUND
13448 
FOUND
13449 
FOUND
13450 
FOUND
13451 
FOUND
13452 
FOUND
13453 
FOUND
13454 
FOUND
13455 
FOUND
13456 
FOUND
13457 
FOUND
13458 
FOUND
13459 
FOUND
13460 
FOUND
13461 
FOUND
13462 
FOUND
13463 
FOUND
13464 
FOUND
13465 
FOUND
13466 
FOUND
13467 
FOUND
13468 
FOUND
13469 
FOUND
13470 
FOUND
13471 
FOUND
13472 
FOUND
13473 
FOUND
13474 
FOUND
13475 
FOUND
13476 
FOUND
13477 
FOUND
13478 
FOUND
13479 
FOUND
13480 
FOUND
13481 
FOUND
13482 
FOUND
13483 
FOUND
13484 
FOUND
13485 
FOUND
13486 
FOUND
13487 
FOUND
13488 
FOUND
13489 

FOUND
14042 
FOUND
14043 
FOUND
14044 
FOUND
14045 
FOUND
14046 
FOUND
14047 
FOUND
14048 
FOUND
14049 
FOUND
14050 
FOUND
14051 
FOUND
14052 
FOUND
14053 
FOUND
14054 
FOUND
14055 
FOUND
14056 
FOUND
14057 
FOUND
14058 
FOUND
14059 
FOUND
14060 
FOUND
14061 
FOUND
14062 
FOUND
14063 
FOUND
14064 
FOUND
14065 
FOUND
14066 
FOUND
14067 
FOUND
14068 
FOUND
14069 
FOUND
14070 
FOUND
14071 
FOUND
14072 
FOUND
14073 
FOUND
14074 
FOUND
14075 
FOUND
14076 
FOUND
14077 
FOUND
14078 
FOUND
14079 
FOUND
14080 
FOUND
14081 
FOUND
14082 
FOUND
14083 
FOUND
14084 
FOUND
14085 
FOUND
14086 
FOUND
14087 
FOUND
14088 
FOUND
14089 
FOUND
14090 
FOUND
14091 
FOUND
14092 
FOUND
14093 
FOUND
14094 
FOUND
14095 
FOUND
14096 
FOUND
14097 
FOUND
14098 
FOUND
14099 
FOUND
14100 
FOUND
14101 
FOUND
14102 
FOUND
14103 
FOUND
14104 
FOUND
14105 
FOUND
14106 
FOUND
14107 
FOUND
14108 
FOUND
14109 
FOUND
14110 
FOUND
14111 
FOUND
14112 
FOUND
14113 
FOUND
14114 
FOUND
14115 
FOUND
14116 
FOUND
14117 
FOUND
14118 

FOUND
14672 
FOUND
14673 
FOUND
14674 
FOUND
14675 
FOUND
14676 
FOUND
14677 
FOUND
14678 
FOUND
14679 
FOUND
14680 
FOUND
14681 
FOUND
14682 
FOUND
14683 
FOUND
14684 
FOUND
14685 
FOUND
14686 
FOUND
14687 
FOUND
14688 
FOUND
14689 
FOUND
14690 
FOUND
14691 
FOUND
14692 
FOUND
14693 
FOUND
14694 
FOUND
14695 
FOUND
14696 
FOUND
14697 
FOUND
14698 
FOUND
14699 
FOUND
14700 
FOUND
14701 
FOUND
14702 
FOUND
14703 
FOUND
14704 
FOUND
14705 
FOUND
14706 
FOUND
14707 
FOUND
14708 
FOUND
14709 
FOUND
14710 
FOUND
14711 
FOUND
14712 
FOUND
14713 
FOUND
14714 
FOUND
14715 
FOUND
14716 
FOUND
14717 
FOUND
14718 
FOUND
14719 
FOUND
14720 
FOUND
14721 
FOUND
14722 
FOUND
14723 
FOUND
14724 
FOUND
14725 
FOUND
14726 
FOUND
14727 
FOUND
14728 
FOUND
14729 
FOUND
14730 
FOUND
14731 
FOUND
14732 
FOUND
14733 
FOUND
14734 
FOUND
14735 
FOUND
14736 
FOUND
14737 
FOUND
14738 
FOUND
14739 
FOUND
14740 
FOUND
14741 
FOUND
14742 
FOUND
14743 
FOUND
14744 
FOUND
14745 
FOUND
14746 
FOUND
14747 
FOUND
14748 

FOUND
15300 
FOUND
15301 
FOUND
15302 
FOUND
15303 
FOUND
15304 
FOUND
15305 
FOUND
15306 
FOUND
15307 
FOUND
15308 
FOUND
15309 
FOUND
15310 
FOUND
15311 
FOUND
15312 
FOUND
15313 
FOUND
15314 
FOUND
15315 
FOUND
15316 
FOUND
15317 
FOUND
15318 
FOUND
15319 
FOUND
15320 
FOUND
15321 
FOUND
15322 
FOUND
15323 
FOUND
15324 
FOUND
15325 
FOUND
15326 
FOUND
15327 
FOUND
15328 
FOUND
15329 
FOUND
15330 
FOUND
15331 
FOUND
15332 
FOUND
15333 
FOUND
15334 
FOUND
15335 
FOUND
15336 
FOUND
15337 
FOUND
15338 
FOUND
15339 
FOUND
15340 
FOUND
15341 
FOUND
15342 
FOUND
15343 
FOUND
15344 
FOUND
15345 
FOUND
15346 
FOUND
15347 
FOUND
15348 
FOUND
15349 
FOUND
15350 
FOUND
15351 
FOUND
15352 
FOUND
15353 
FOUND
15354 
FOUND
15355 
FOUND
15356 
FOUND
15357 
FOUND
15358 
FOUND
15359 
FOUND
15360 
FOUND
15361 
FOUND
15362 
FOUND
15363 
FOUND
15364 
FOUND
15365 
FOUND
15366 
FOUND
15367 
FOUND
15368 
FOUND
15369 
FOUND
15370 
FOUND
15371 
FOUND
15372 
FOUND
15373 
FOUND
15374 
FOUND
15375 
FOUND
15376 

FOUND
15933 
FOUND
15934 
FOUND
15935 
FOUND
15936 
FOUND
15937 
FOUND
15938 
FOUND
15939 
FOUND
15940 
FOUND
15941 
===NOT FOUND===
15942 
FOUND
15943 
FOUND
15944 
FOUND
15945 
FOUND
15946 
FOUND
15947 
FOUND
15948 
FOUND
15949 
FOUND
15950 
FOUND
15951 
FOUND
15952 
FOUND
15953 
FOUND
15954 
FOUND
15955 
FOUND
15956 
FOUND
15957 
FOUND
15958 
FOUND
15959 
FOUND
15960 
FOUND
15961 
FOUND
15962 
FOUND
15963 
FOUND
15964 
FOUND
15965 
FOUND
15966 
FOUND
15967 
FOUND
15968 
FOUND
15969 
FOUND
15970 
FOUND
15971 
FOUND
15972 
FOUND
15973 
FOUND
15974 
FOUND
15975 
FOUND
15976 
FOUND
15977 
FOUND
15978 
FOUND
15979 
FOUND
15980 
FOUND
15981 
FOUND
15982 
FOUND
15983 
FOUND
15984 
FOUND
15985 
FOUND
15986 
FOUND
15987 
FOUND
15988 
FOUND
15989 
FOUND
15990 
FOUND
15991 
FOUND
15992 
FOUND
15993 
FOUND
15994 
FOUND
15995 
FOUND
15996 
FOUND
15997 
FOUND
15998 
FOUND
15999 
FOUND
16000 
FOUND
16001 
FOUND
16002 
FOUND
16003 
FOUND
16004 
FOUND
16005 
FOUND
16006 
FOUND
16007 
FOUND
16008 
FO

In [ ]:
for name, df in AMENITIES.items():
    print(f'\n\n{name}')
    display(df.head(3))

### Find count of each amenity within each tract

In [ ]:
def within_tract_counts(name, df):
    within_geoid_counts = df.groupby(f'{name}_GEOID_MATCH')[f'{name}_GEOID_MATCH'].count()
    print(within_geoid_counts.min(), within_geoid_counts.max())
    within_geoid_counts.name = f'N_{name}_WITHIN_TRACT'
    within_geoid_counts = within_geoid_counts.reset_index()
    return within_geoid_counts

AMENITY_WITHIN_TRACT_COUNTS = {}
for name, df in AMENITIES.items():
    AMENITY_WITHIN_TRACT_COUNTS[name] = within_tract_counts(name, df)
    print(f'\n\n{name}')
    display(AMENITY_WITHIN_TRACT_COUNTS[name].head(3))

Append the results and perform check.

In [ ]:
for name, df in AMENITY_WITHIN_TRACT_COUNTS.items():
    result = result.merge(df, how='left', left_on='GEOID', right_on=f'{name}_GEOID_MATCH')
result.head()

Check if number of unique geoids same in each table.

In [ ]:
for name, df in AMENITY_WITHIN_TRACT_COUNTS.items():
    print(result[f'{name}_GEOID_MATCH'].nunique() ==\
          AMENITY_WITHIN_TRACT_COUNTS[name][f'{name}_GEOID_MATCH'].nunique())

Fill missing data with 0 (i.e. no stores found within that tract).

Drop the GEOID_MATCH field. No longer needed.

In [ ]:
for col in [x for x in result.columns if x.startswith('N_')]:
    result[col] = result[col].fillna(0)
    result = result.drop(columns=[f'{col.split('_')[1]}_GEOID_MATCH'])
result

The following blocks of commented code were a first attempt to calculate stores that fell within a certain radius of a census tract center. This would have taken far too long to compute (~5 days).

In [ ]:
### Now, compute the stores within radius of neighborhood centers
# N_GROCERY_WT_2_MI, N_GROCERY_WT_5_MI, N_GROCERY_WT_30_MI 


In [ ]:
# from sklearn.metrics.pairwise import haversine_distances
# from math import radians

# # example
# bsas = [-34.83333, -58.5166646]
# paris = [49.0083899664, 2.53844117956]
# bsas_in_radians = [radians(_) for _ in bsas]
# paris_in_radians = [radians(_) for _ in paris]
# r = haversine_distances([bsas_in_radians, paris_in_radians])
# r = (r * 6371000/1000) # multiply by Earth radius to get kilometers
# r = (r * 0.621371)# km to mile
# r

In [ ]:
# # Since doing many pairwise calcs, prep all items before the distance calcs are performed
# # will parallelize the distance calcs

# tract_prep = [] # store (i, prepared_location)
# # iterate through each tract
# for i, row in result.iterrows():
#     tract_lat = row.INTPTLAT
#     tract_lon = row.INTPTLONG
#     tract_in_rads = [radians(x) for x in [tract_lat, tract_lon]]
#     tract_prep.append((i, tract_in_rads))

# amen_prep = []
# for j, rowj in gro.iterrows():
#     am_lat = rowj.lat_cleaned
#     am_lon = rowj.lon_cleaned
#     am_in_rads = [radians(x) for x in [am_lat, am_lon]]
#     amen_prep.append((j, am_in_rads))
    
# print(len(tract_prep), len(amen_prep))

In [ ]:
# iterator = ((x,y) for x in tract_prep for y in amen_prep) # build iterator
# distances = []
# count = 1.0
# total = len(tract_prep) * len(amen_prep)

# for a, b in iterator:
#     print('{}%'.format(round(count/total*100., 2), end=''))
#     i = a[0]
#     j = b[0]
#     r = haversine_distances([a[1], b[1]])[0][1] * 3958.754641
#     count += 1

### Calculate which stores (and number of stores) fall within some distance of the tract centers.

Create a transformer to project coordinates from latitude/longitude to a 2D plane of the USA.

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("epsg:4326", "epsg:2163") # lat/lon to us 2d projection

Project all of the tract locations. Store for use below.

In [ ]:
# transform coordinate system
tract_prep = [] # store (i, prepared_location)
# iterate through each tract
for i, row in result.iterrows():
    print('\r{} of {}'.format(i+1, result.shape[0]), end='')
    tract_lat = row.INTPTLAT
    tract_lon = row.INTPTLONG
    x, y = transformer.transform(tract_lat, tract_lon)
    point = [x, y]
    tract_prep.append((i, point))
print('tracts complete\n')

Project all of the amenities locations for use below.

In [ ]:
def amenity_projections(df):
    amen_prep = []
    for j, rowj in df.iterrows():
        print('\r{} of {}'.format(j+1, df.shape[0]), end='')
        am_lat = rowj.lat_cleaned
        am_lon = rowj.lon_cleaned
        x, y = transformer.transform(am_lat, am_lon)
        point = [x, y]
        amen_prep.append((j, point))
    return amen_prep

AMENITIES_PROJ = {}
for name, df in AMENITIES.items():
    print(f'\n\nProjecting {name}')
    AMENITIES_PROJ[name] = amenity_projections(df)
    print(f'\n{name}: {len(AMENITIES_PROJ[name])}')
print(f'Tracts: {len(tract_prep)}')

Build a kd-tree of the projected amenities points.

In [ ]:
def amenity_kdtree(amenity_projection_list):
    points = np.array([x[1] for x in amenity_projection_list])
    print(points)
    point_tree = spatial.cKDTree(points)
    return point_tree   

AMENITY_KD_TREES = {}
for name, amenity_projection_list in AMENITIES_PROJ.items():
    print(name)
    AMENITY_KD_TREES[name] = amenity_kdtree(amenity_projection_list)

Iterate through the projected tract coordinates. For each amenity, query its kdtree with these coordinates and a distance. The units produced by the projection are in meters. We specify the number of miles to the miles_to_meters function to convert it to meters.

This code stores a list of amenity indexes that fall within the radius of the census tract center. These lists are later saved to the final 'full' table.

The number in these column names specifies the number of miles that were searched (radius) from the tract centers.

In [ ]:
def miles_to_meters(miles):
    meters = miles * 1609.34
    return meters

def meters_to_miles(meters):
    miles = meters / 1609.34
    return miles

def get_distance_results(tract_prep, point_tree):
    # for each tract, get a list of amenity indexes that fall within x miles 
    distance_results = defaultdict(list)
    for result_idx, locationxy in tract_prep:
        for dist in [2,5,10,25,50]: #miles
            point_list = point_tree.query_ball_point(locationxy, miles_to_meters(dist), p=np.inf)
            n_points = len(point_list)
            distance_results['dist{}'.format(dist)].append(point_list)
    return distance_results

AMENITY_DISTANCE_RESULTS = {}
for name, point_tree in AMENITY_KD_TREES.items():
    print(f'{name}')
    AMENITY_DISTANCE_RESULTS[name] = pd.DataFrame(get_distance_results(tract_prep, point_tree))
    display(AMENITY_DISTANCE_RESULTS[name].head(3))

Create weighted number for amenities within x distance

Weights go as 1 / distance (radius distance, in miles)

In [ ]:
for name, df in AMENITY_DISTANCE_RESULTS.items():
    for col in df.columns:
        weight = 1. / int(col[4:])
        new_col = f'wt_n_{name}_dist_{col[4:]}'.upper()
        df[new_col] =  df[col].apply(lambda x: len(x)) * weight
        AMENITY_DISTANCE_RESULTS[name] = df
        print(name)
        display(AMENITY_DISTANCE_RESULTS[name].head(3))

Update the dist field names.    

In [ ]:
for name, df in AMENITY_DISTANCE_RESULTS.items():
    dist_cols = [x for x in df.columns if x.startswith('dist')]
    for col in dist_cols:
        df.rename(columns={col: f'LIST_{name}_DIST_{col[4:]}'}, inplace=True)
    print(name)
    display(AMENITY_DISTANCE_RESULTS.head(3))

Append the newly calc'd info to the `result` table

In [ ]:
for name, df in AMENITY_DISTANCE_RESULTS.items():
    result = pd.concat([result, df], axis=1)
result.head(3)

Output this full table to a file. This is rather large. It is needed for the visualization but not for clustering.

In [ ]:
OUTPATH_ROOT = '../../amenities'
FULL_NAME = 'amenities_full.pkl.gz'

result.to_pickle(os.path.join(OUTPATH_ROOT, FULL_NAME), protocol=4, compression='gzip')

Output a much smaller version (per disk storage) of the table without the list columns. This would be ideal to use for clustering.

In [ ]:
FEAT_NAME = 'amenities_features.pkl'
result[[x for x in result.columns if not x.startswith('LIST_')]].to_pickle(
    os.path.join(OUTPATH_ROOT, FEAT_NAME), protocol=4
)

Test read each output

In [ ]:
print(FULL_NAME)
display(pd.read_pickle(os.path.join(OUTPATH_ROOT, FULL_NAME), compression='gzip').head())
print(FEAT_NAME)
display(pd.read_pickle(os.path.join(OUTPATH_ROOT, FEAT_NAME)).head())